<a href="https://colab.research.google.com/github/Sugirlstar/GPU_griddedCalcualtion_draft/blob/main/SPI_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

挂载Google Driver：

1、为什么要挂载Google Driver：如果没有挂载Google Driver（Google 云端硬盘），则下载的文件只会暂时存在（即下次打开时，下载在Jupyter NoteBook的文件就不见了），初始阶段，左侧只有sample_data一个文件夹，当使用代码挂载Google Driver后，左侧出现driver文件夹，只要我们的文件放在driver后，文件就不会消失了（下次打开的时候需要再次使用代码挂载Google Driver）

在将 nc 文件上传到 Google Drive 之后，你可以使用以下步骤在 Google Colab 中挂载你的 Google Drive，然后读取和处理这些 nc 文件。

该代码将打开一个新窗口，要求你登录 Google 账户并授权 Colab 访问你的 Google Drive。操作完成后，你将获得一个授权代码。将此代码复制并粘贴回 notebook，然后按 Enter。

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


本质是一个linux虚拟机：在左侧文件列表中返回上一级菜单，可以看到linux文件结构

In [2]:
!pip install netCDF4

In [3]:
!pip install xarray

In [4]:
import os
os.environ['HDF5_USE_FILE_LOCKING'] = 'FALSE'

In [5]:
# 安装cuda
!curl https://colab.chainer.org/install | sh -


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1553  100  1553    0     0   6793      0 --:--:-- --:--:-- --:--:--  6811
********************************************************************************
CUDA version could not be detected!
Try installing Chainer manually by: pip install chainer
********************************************************************************


In [6]:
import netCDF4 as nc
import numpy as np
import cupy as cp
from scipy.stats import gamma
import xarray as xr
import time

In [7]:
# 指定 Google Drive 中的 nc 文件路径
file_paths = [f"/content/drive/MyDrive/ColabNotebooks/daily_{year}.nc" for year in range(1990, 2001)]
# 创建一个空列表用于存储所有数据
all_data = []

# 遍历文件列表
for file in file_paths:
    # 打开netCDF文件
    dataset = nc.Dataset(file)
    # 获取tp变量的数据
    tp_data = dataset.variables['tp'][:]
    all_data.append(tp_data)

In [8]:
precip_data = np.concatenate(all_data)
precip_data = np.array(precip_data, dtype='float32')
precip_data.shape


(4018, 361, 641)

In [9]:
##### 生成日期序列的函数
from datetime import datetime, timedelta
def generate_date_sequence(start_date_str, end_date_str, date_format="%Y%m%d"):
    # 将开始日期和结束日期转换为datetime对象
    start_date = datetime.strptime(start_date_str, date_format)
    end_date = datetime.strptime(end_date_str, date_format)

    # 创建一个空列表用于存储日期序列
    date_sequence = []

    # 初始化当前日期为开始日期
    current_date = start_date

    # 遍历日期范围，将每个日期添加到序列中
    while current_date <= end_date:
        date_sequence.append(current_date.strftime(date_format))
        # 递增当前日期
        current_date +=timedelta(days=1)

    return date_sequence


##### 判断是否为闰年的函数
def is_leap_year(year):
    """
    判断给定年份是否为闰年。
    """
    return (year % 4 == 0 and year % 100 != 0) or (year % 400 == 0)

##### 根据日期序列生成daylist的函数
def compute_dlist_from_sequence(date_sequence):
    # 创建一个长度为365的列表，使用空列表初始化每个元素
    dlist = [[] for _ in range(365)]

    # 遍历日期序列
    for idx, date_str in enumerate(date_sequence):
        date_obj = datetime.strptime(date_str, "%Y%m%d")

        # 获取一年中的第几天 (1-366)
        day_of_year = date_obj.timetuple().tm_yday

        # 对于2月29日，调整为2月28日
        if date_obj.month == 2 and date_obj.day == 29:
            dlist[58].append(idx)  # 58是非闰年中2月28日的位置
        else:
            # 调整为闰年
            position = day_of_year - 1
            if day_of_year > 59 and is_leap_year(date_obj.year):
                position -= 1
            dlist[position].append(idx)

    return dlist


# 生成日期序列
date_sequence = generate_date_sequence("19900101", "20001231")
# 根据日期序列生成daylist
daylist = compute_dlist_from_sequence(date_sequence)

In [10]:
# CPU computation
def compute_90_day_cumulative_cpu(data):
    """
    Compute the 90-day cumulative values for a 3D CuPy array.

    Args:
    - data (cp.ndarray): A 3D CuPy array with the shape (days, longitude, latitude).

    Returns:
    - cp.ndarray: A 3D CuPy array with the shape (days, longitude, latitude) containing the 90-day cumulative values.
    """

    result = []
    for i in range(89, data.shape[0]):
        # Sum the values from day t-89 to day t
        moving_sum = np.sum(data[i-89:i+1], axis=0)
        result.append(moving_sum)

    return np.stack(result)

start_time_cpu = time.time()

# Compute the 90-day cumulative values for the data
cumulative_90_days_cpu = compute_90_day_cumulative_cpu(precip_data)

end_time_cpu = time.time()

cpu_time = end_time_cpu - start_time_cpu
print(cpu_time)

47.72395443916321


In [11]:
# GPU computation
def compute_90_day_cumulative_cupy(data):
    """
    Compute the 90-day cumulative values for a 3D CuPy array.

    Args:
    - data (cp.ndarray): A 3D CuPy array with the shape (days, longitude, latitude).

    Returns:
    - cp.ndarray: A 3D CuPy array with the shape (days, longitude, latitude) containing the 90-day cumulative values.
    """

    result = []
    for i in range(89, data.shape[0]):
        # Sum the values from day t-89 to day t
        moving_sum = cp.sum(data[i-89:i+1], axis=0)
        result.append(moving_sum)

    return cp.stack(result)


# Transfer data from CPU to GPU
start_time_gpu = time.time()
precip_data_gpu = cp.asarray(precip_data)

# Compute the 90-day cumulative values for the data
cumulative_90_days_cupy = compute_90_day_cumulative_cupy(precip_data_gpu)

# Convert the result back to a NumPy array
cumulative_90_days_numpy = cp.asnumpy(cumulative_90_days_cupy)

end_time_gpu = time.time()
gpu_time = end_time_gpu - start_time_gpu
print(gpu_time)

6.433187007904053
